In [1]:
library(tidyverse)
library(lme4)
library(ggplot2)

-- Attaching packages --------------------------------------- tidyverse 1.2.1 --

v ggplot2 3.2.1     v purrr   0.3.3
v tibble  2.1.3     v dplyr   0.8.3
v tidyr   1.0.0     v stringr 1.4.0
v readr   1.3.1     v forcats 0.4.0

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()

Loading required package: Matrix


Attaching package: 'Matrix'


The following objects are masked from 'package:tidyr':

    expand, pack, unpack




In [2]:
df_men <- read_csv("../data/temp_file_for_level_adj_men.csv")
df_women <- read_csv("../data/temp_file_for_level_adj_women.csv")


Parsed with column specification:
cols(
  Athlete_name = col_character(),
  Adjusted_score = col_double(),
  Season = col_double(),
  tier = col_double()
)

Parsed with column specification:
cols(
  Athlete_name = col_character(),
  Adjusted_score = col_double(),
  Season = col_double(),
  tier = col_double()
)



In [3]:
df_men['tier'] <- lapply(df_men['tier'], as.character)
df_men['Season'] <- lapply(df_men['Season'], as.character)


In [4]:
head(df_men)

Athlete_name,Adjusted_score,Season,tier
<chr>,<dbl>,<chr>,<chr>
COTA Jeremy - USA,107.983768,2010,1
DIGRAVIO David - USA,113.135247,2010,1
DISCOE Jimmy - USA,11.890073,2010,1
MARQUIS Philippe - CAN,106.418499,2010,1
ROCHON Cedric - CAN,106.396707,2010,1
DIGRAVIO David - USA,9.971327,2010,1


In [4]:
model_season <- lmer(Adjusted_score ~ tier + (1 | Season), data = df_men)
summary(model_season)

Linear mixed model fit by REML ['lmerMod']
Formula: Adjusted_score ~ tier + (1 | Season)
   Data: df_men

REML criterion at convergence: 5159.5

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-1.78822 -0.91112  0.01147  0.84977  2.72251 

Random effects:
 Groups   Name        Variance Std.Dev.
 Season   (Intercept)  187.6   13.70   
 Residual             2022.2   44.97   
Number of obs: 494, groups:  Season, 10

Fixed effects:
            Estimate Std. Error t value
(Intercept)   74.673      5.531  13.502
tier2        -21.468      4.559  -4.708
tier3        -27.862      6.806  -4.094

Correlation of Fixed Effects:
      (Intr) tier2 
tier2 -0.431       
tier3 -0.270  0.398

In [53]:
model_season_id <- lmer(Adjusted_score ~ tier + (1 | Season) + (1|Athlete_name), data = df_men)
summary(model_season_id)


Linear mixed model fit by REML ['lmerMod']
Formula: Adjusted_score ~ tier + (1 | Season) + (1 | Athlete_name)
   Data: df_men

REML criterion at convergence: 5159.4

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-1.75690 -0.90223  0.00634  0.85270  2.70110 

Random effects:
 Groups       Name        Variance Std.Dev.
 Athlete_name (Intercept)   13.71   3.703  
 Season       (Intercept)  176.33  13.279  
 Residual                 2011.14  44.846  
Number of obs: 494, groups:  Athlete_name, 65; Season, 10

Fixed effects:
            Estimate Std. Error t value
(Intercept)   74.649      5.466   13.66
tier2        -21.362      4.565   -4.68
tier3        -27.508      6.826   -4.03

Correlation of Fixed Effects:
      (Intr) tier2 
tier2 -0.439       
tier3 -0.280  0.401

In [54]:
anova(model_season, model_season_id)

refitting model(s) with ML (instead of REML)



,Df,AIC,BIC,logLik,deviance,Chisq,Chi Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
model_season,5,5184.794,5205.807,-2587.397,5174.794,NA,NA,NA
model_season_id,6,5186.660,5211.875,-2587.330,5174.660,0.1348592,1,0.7134461


In [55]:
df_men['predicted_Scores'] <- predict(model_season, 
        newdata = df_men,
        allow.new.levels = TRUE)

In [5]:
df_men %>% group_by(Season, tier) %>% summarize(avg_scores = mean(Adjusted_score))

Season,tier,avg_scores
<chr>,<chr>,<dbl>
2010,1,68.04989
2010,2,43.32131
2011,1,60.72697
2011,2,76.27265
2012,1,58.30413
2012,2,34.20130
2013,1,46.74901
2013,2,67.90092
2014,1,58.59349


In [6]:
model_coef <- coef(model_season)$Season

In [7]:
model_coef$tier2[1]

[1] -21.46763

In [9]:
df_all_men <- read_csv("../data/all_tiers_adjusted_scores_men.csv")
df_all_women <- read_csv("../data/all_tiers_adjusted_scores_women.csv")


Parsed with column specification:
cols(
  Athlete_Comp_ID = col_character(),
  Athlete_name = col_character(),
  Score_Metric = col_double(),
  Finish_Rank = col_double(),
  mean = col_double(),
  stddev = col_double(),
  Score_Multiplier = col_double(),
  Adjusted_score = col_double(),
  Season = col_double(),
  Season_Comp = col_character(),
  Round = col_character(),
  tier = col_double()
)

Parsed with column specification:
cols(
  Athlete_Comp_ID = col_character(),
  Athlete_name = col_character(),
  Score_Metric = col_double(),
  Finish_Rank = col_double(),
  mean = col_double(),
  stddev = col_double(),
  Score_Multiplier = col_double(),
  Adjusted_score = col_double(),
  Season = col_double(),
  Season_Comp = col_character(),
  Round = col_character(),
  tier = col_double()
)



In [10]:
df_all_men <- df_all_men %>% mutate(lvl_adj = case_when(tier == 1 ~ 0, tier == 2 ~ model_coef$tier2[1], tier == 3 ~ model_coef$tier3[1]))
df_all_women <- df_all_women %>% mutate(lvl_adj = case_when(tier == 1 ~ 0, tier == 2 ~ model_coef$tier2[1], tier == 3 ~ model_coef$tier3[1]))

df_all_men <- df_all_men %>% mutate(final_adj_score = Adjusted_score - lvl_adj)
df_all_women <- df_all_women %>% mutate(final_adj_score = Adjusted_score - lvl_adj)

In [11]:
write_csv(df_all_men, "../data/final_all_tiers_adjusted_scores_men.csv")
write_csv(df_all_women, "../data/final_all_tiers_adjusted_scores_women.csv")